In [1]:
!pip install -q transformers

In [2]:

!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.8 MB/s eta 0:00:00


In [3]:

#!pip3 uninstall --yes torch torchaudio torchvision torchtext torchdata
!pip3 install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [4]:
import os
import math
import numpy as np
import random
import logging

# Bring in PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
# Most of the examples have typing on the signatures for readability
from typing import Optional, Callable, List, Tuple
from Bio import SeqIO
# For data loading
from torch.utils.data import Dataset, IterableDataset, TensorDataset, DataLoader
import json
import glob
import gzip
import bz2

# For progress and timing
from tqdm import tqdm
import time
import shutil
from Bio.PDB import PDBList
from Bio.PDB.MMCIFParser import MMCIFParser
import re

#phi and psi
from Bio.PDB import PICIO, PDBIO
from Bio import PDB
from typing import TypedDict, Dict, Tuple

In [12]:
from Bio import Entrez
from Bio.PDB import PDBList
import pandas as pd

def download_pdb_files(pdb_ids, output_dir):
  pdbl = PDBList()
  for pdb_id in pdb_ids:
    pdb_id_trimmed = pdb_id[:-1] #for some reason the last letter is random, we have to remove it to retrieve the ids
    pdbl.retrieve_pdb_file(pdb_id_trimmed, pdir=output_dir, file_format='pdb')

file_path = '/content/cullpdb_pc30.0_res0.0-2.0_noBrks_noDsdr_len40-10000_R0.25_Nmr_d2024_06_13_chains2636'
df = pd.read_csv(file_path, delim_whitespace=True)

#pdb_ids = df['PDBchain'].tolist()

df = df[df['len'] <= 128]
pdb_ids = df['PDBchain'].tolist()
print(pdb_ids)

download_pdb_files(pdb_ids, '/content/target directory')

['1E0ZA', '1RY4A', '1UG7A', '1WHNA', '2AYAA', '2D87A', '2DB5A', '2DKPA', '2EBKA', '2ETTA', '2FM4A', '2I4KA', '2JQ5A', '2K0RA', '2K5TA', '2LR4A', '2LS01', '2LVLA', '2MAHA', '2NCNA', '2RLOA', '2RV8A', '2YTUA', '4AR0A', '5AIWA', '5H7UA', '5N9QA', '8E22X', '1CMOA', '1DDFA', '1EALA', '1PQNA', '1QKLA', '1UEYA', '1W2QA', '1WH4A', '1WJRA', '1X5MA', '1ZU1A', '2CSOA', '2DMKA', '2EE7A', '2L1LB', '2LOEA', '2MF7A', '2MPLA', '2NBOA', '2TMPA', '4UEIA', '5M1UA', '7XRWA', '8OILA', '1RSFA', '1V5SA', '1X5EA', '1ZO0A', '2A4HA', '2CR4A', '2DAVA', '2IF1A', '2JP2A', '2JVNA', '2K2VA', '2L3GA', '2L57A', '2LBCA', '2LLNA', '2M26A', '3MSPA', '7F7NA', '7KNVA', '8SQXA', '1BUQA', '1BW3A', '1FR0A', '1GIOA', '1MH6A', '1PSYA', '1UENA', '1WYJA', '2AI6A', '2DHJA', '2K4VA', '2K4ZA', '2KA5A', '2KFPA', '2KYTA', '2L2CA', '2LDUA', '2LM0A', '2MDKA', '2MV4A', '2N2HB', '2NBGA', '2PRFA', '4A53A', '5KNWA', '5T3YA', '5WOEA', '6GWMA', '6TH8A', '6UF2A', '6XEHA', '7BQCA', '7PRDA', '7QYIA', '1DC7A', '1E8EA', '1EO1A', '1PIRA', '1T3VA', 

In [13]:
from transformers import BertModel, BertTokenizer
import re

In [14]:
len(df)

1723

In [30]:
from Bio.PDB import PDBParser

def extract_input(pdb_file):
    parser = PDBParser()
    structure = parser.get_structure("protein", pdb_file)
    model0 = structure[0]
    chain_A = model0['A']  # and we get chain A
    # dictionary converting 3-letter codes to 1-letter codes
    # this is a very common need in bioinformatics of proteins
    d3to1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
    'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N',
    'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W',
    'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

    sequence = []
    for residue in chain_A:
        # for simplicity we can use X for heteroatoms (ions and water)
        sequence.append(d3to1.get(residue.get_resname(), 'X'))  #converts water and ions to X
    #print(''.join(sequence))
    seq_example =  ' '.join(sequence)
    #return seq_example

#def extract_coordinates(pdb_file):
    structure.atom_to_internal_coordinates() # turns xyz coordinates into angles and bond lengths

    chain:PDB.Chain.Chain = list(structure.get_chains())[0]#iterator of chains, turns it into list, [0] first chain

    ic_chain: PDB.internal_coords.IC_Chain = chain.internal_coord #this access the internal chain coords of the chain object

    d: Dict[Tuple[PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey],
        PDB.internal_coords.Dihedron] = ic_chain.dihedra

    cnt = 1
    phi_angles = {}
    phi_angles_list = []
    psi_angles = {}
    psi_angles_list = []

    for key in d:
      if key[0].akl[3] == 'N' and key[1].akl[3] == 'CA' and key[2].akl[3] == 'C' and key[3].akl[3] == 'N':
          phi_angles[key] = d[key].angle
          phi_angles_list.append(d[key].angle)
      elif key[0].akl[3] == 'CA' and key[1].akl[3] == 'C' and key[2].akl[3] == 'N' and key[3].akl[3] == 'CA':
          psi_angles[key] = d[key].angle
          psi_angles_list.append(d[key].angle)


    structure.internal_to_atom_coordinates(verbose = True)
    io = PDBIO() #this is to write a pdb file again
    io.set_structure(structure)#set structure, the structure you wan tin the pdb file
    phi_angles_list.append(0)
    psi_angles_list.append(0)

    phi = np.asarray(phi_angles_list,dtype=np.float32)
    psi = np.asarray(psi_angles_list,dtype=np.float32)
    angles = np.vstack((psi,phi))

    tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
    seq = re.sub(r"[UZOB]", "X", seq_example)
    encoded_input = tokenizer(seq, return_tensors='pt')
    output = bert_model(**encoded_input)
    embedded_pb = output.last_hidden_state
    N, D = embedded_pb.size()[1], embedded_pb.size()[2]
    embeddings = embedded_pb
    print('N:', N,D)
    return seq_example, angles, encoded_input


In [22]:
class SelfAttention(nn.Module):
    def __init__(self, embed_dim: int):
        super(SelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.w_q = nn.Parameter(torch.randn(embed_dim, embed_dim))
        self.w_k = nn.Parameter(torch.randn(embed_dim, embed_dim))
        self.w_v = nn.Parameter(torch.randn(embed_dim, embed_dim))

    def forward(self, embeddings_prot_bert: torch.Tensor) -> torch.Tensor:
        Q = torch.matmul(embeddings_prot_bert, self.w_q)
        K = torch.matmul(embeddings_prot_bert, self.w_k)
        V = torch.matmul(embeddings_prot_bert, self.w_v)

        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(K.size(-1))
        attn = torch.softmax(scores, dim=-1)
        attention_output = torch.matmul(attn, V)

        return attention_output

class TransformerModel(nn.Module):
    def __init__(self, embed_dim: int, feed_forward_dim1: int, feed_forward_dim2: int, output_dim: int = 2, dropout_rate: float = 0.1):
        super(TransformerModel, self).__init__()
        self.self_attention = SelfAttention(embed_dim)
        self.layer_norm1 = nn.LayerNorm(embed_dim)
        self.layer_norm2 = nn.LayerNorm(output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_dim, feed_forward_dim1),
            nn.GELU(),
            self.dropout,
            nn.Linear(feed_forward_dim1, feed_forward_dim2),
            nn.GELU(),
            self.dropout,
            nn.Linear(feed_forward_dim2, output_dim)
        )

    def forward(self, embeddings: torch.Tensor) -> torch.Tensor:
        attention_output = self.self_attention(embeddings)
        normalized_attention_output = self.layer_norm1(attention_output)
        ff_output = self.feed_forward(normalized_attention_output)
        output = self.layer_norm2(ff_output)
        return ff_output
class CustomLoss(nn.Module):
    def __init__(self, predicted_angles, angles_tensor):
        super(CustomLoss, self).__init__()
        self.predicted_angles= predicted_angles
        self.angles_tensor= angles_tensor
    def forward(self):
        print(self.predicted_angles)
        print(self.angles_tensor)
        d_list = []
        for i in range(len(self.angles_tensor)):
            x1, y1 = self.predicted_angles[0][i]
            x2, y2 = self.angles_tensor[i]
            ax_x = torch.min(torch.abs(x2 - x1), torch.abs(360 - torch.abs(x2 - x1)))
            ax_y = torch.min(torch.abs(y2 - y1), torch.abs(360 - torch.abs(y2 - y1)))
            d = torch.sqrt(ax_x**2 + ax_y**2)
            d_list.append(d)
        d_tensor = torch.stack(d_list)
        return d_tensor.mean()
class TransformerTrainer:
    def __init__(self, model: nn.Module, CustomLoss: nn.Module, num_epochs: int, sequence: torch.Tensor, angles: torch.Tensor):
        self.model = model
        self.CustomLoss = CustomLoss
        self.num_epochs = num_epochs
        self.sequence = sequence
        self.angles_tensor = angles
        self.optimizer = optim.SGD(model.parameters(), lr=0.001)
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.1)

    def train(self):
        for epoch in range(self.num_epochs):
            self.optimizer.zero_grad()
            predictions = self.model.forward(self.sequence)[:, :len(self.angles_tensor)]
            print(self.angles_tensor)
            loss = CustomLoss()
            loss.forward(torch.tensor(predictions, requires_grad=True), self.angles_tensor)
            #loss = self.criterion(predictions.squeeze(), self.angles_tensor)
            loss.backward(retain_graph=True)
            self.optimizer.step()
            #self.scheduler.step()
            #loss_list.append(loss.item())
            print(loss)
            print(f"Epoch {epoch + 1}/{self.num_epochs}, Loss: {loss.item()}")
        return loss


In [23]:
#with  using mean angle error loss directly
class TransformerTrainer2:
    def __init__(self, model: nn.Module, criterion: nn.Module, num_epochs: int, sequence: torch.Tensor, angles: np.ndarray):
        self.model = model
        self.criterion = criterion
        self.num_epochs = num_epochs
        self.sequence = sequence
        self.angles_tensor = torch.from_numpy(angles.T)
        self.optimizer = optim.SGD(model.parameters(), lr=0.005)

    def train(self):
        loss_list = []
        for epoch in range(self.num_epochs):
            self.optimizer.zero_grad()
            predictions = self.model.forward(self.sequence)[:, :len(self.angles_tensor)]

            loss = self.criterion(predictions.squeeze(), self.angles_tensor)


            loss.backward(retain_graph=True)
            #for name, param in self.model.named_parameters():
            #    if param.grad is not None:
            #        print(f"Gradient for {name}: {param.grad.norm()}")

            self.optimizer.step()
            loss_list.append(loss.item())

            print(f"Epoch {epoch + 1}/{self.num_epochs}, Loss: {loss.item()}")

class MeanAngleErrorLoss(nn.Module):
    def __init__(self):
        super(MeanAngleErrorLoss, self).__init__()

    def forward(self, predictions, angles):

        angular_diff_phi = torch.abs(predictions[:, 0] - angles[:, 0])
        alt_diff_phi = 2 * torch.pi - angular_diff_phi
        error_phi = torch.min(angular_diff_phi, alt_diff_phi)
        #print(predictions)

        angular_diff_psi = torch.abs(predictions[:, 1] - angles[:, 1])
        alt_diff_psi = 2 * torch.pi - angular_diff_psi
        error_psi = torch.min(angular_diff_psi, alt_diff_psi)


        error_combined = torch.stack((error_phi, error_psi), dim=1)
        #print(np.shape(error_combined))
        mse = torch.mean(error_combined ** 2)
        #maybe we can use squeeze here
        return mse

In [24]:
#with  using mean angle error loss with mse loss
class TransformerTrainer3:
    def __init__(self, model: nn.Module, criterion: nn.Module, num_epochs: int, sequence: torch.Tensor, angles: np.ndarray):
        self.model = model
        self.criterion = criterion
        self.num_epochs = num_epochs
        self.sequence = sequence
        self.angles_tensor = torch.from_numpy(angles.T).float()
        self.optimizer = optim.SGD(model.parameters(), lr=0.005)
        self.empty_tensor = torch.zeros_like(self.angles_tensor)

    def train(self):
        loss_list = []
        for epoch in range(self.num_epochs):
            self.optimizer.zero_grad()
            predictions = self.model.forward(self.sequence)[:, :len(self.angles_tensor)]
            mean_angle_error = MeanAngleErrorLoss2()(predictions.squeeze(), self.angles_tensor)
            loss = self.criterion(mean_angle_error, self.empty_tensor)


            loss.backward(retain_graph=True)
            self.optimizer.step()
            loss_list.append(loss.item())

            print(f"Epoch {epoch + 1}/{self.num_epochs}, Loss: {loss.item()}")

class MeanAngleErrorLoss2(nn.Module):
    def __init__(self):
        super(MeanAngleErrorLoss2, self).__init__()


    def forward(self, predictions, angles):

        angular_diff_phi = torch.abs(predictions[:, 0] -angles[:, 0])
        alt_diff_phi = 2 * torch.pi - angular_diff_phi
        error_phi = torch.min(angular_diff_phi, alt_diff_phi)

        angular_diff_psi = torch.abs(predictions[:, 1] - angles[:, 1])
        alt_diff_psi = 2 * torch.pi - angular_diff_psi
        error_psi = torch.min(angular_diff_psi, alt_diff_psi)

        error_combined = torch.stack((error_phi, error_psi), dim=1)
        #print(np.shape(error_combined))
        #mse = torch.mean(error_combined ** 2)
        #maybe we can use squeeze here
        return error_combined

In [25]:
#checking whats wrong with  angle loss, by checking gradients and scaling to mse loss
class TransformerTrainer4:
    def __init__(self, model: nn.Module, criterion: nn.Module, num_epochs: int, sequence: torch.Tensor, angles: np.ndarray):
        self.model = model
        self.criterion = criterion
        self.num_epochs = num_epochs
        self.sequence = sequence
        self.angles_tensor = torch.from_numpy(angles.T)
        self.optimizer = optim.SGD(model.parameters(), lr=0.005)

    def train(self):
        loss_list = []
        for epoch in range(self.num_epochs):
            self.optimizer.zero_grad()
            predictions = self.model.forward(self.sequence)[:, :len(self.angles_tensor)]

            #loss = self.criterion(predictions.squeeze(), self.angles_tensor)


            #loss.backward(retain_graph=True)
            custom_loss = self.criterion(predictions.squeeze(), self.angles_tensor)
            scaled_custom_loss = 2 * custom_loss

            # Compute MSE loss for comparison
            mse_loss = nn.MSELoss()(predictions.squeeze(), self.angles_tensor)
            scaled_custom_loss = scaled_custom_loss + mse_loss

            print(f"Epoch {epoch + 1}/{self.num_epochs}, Scaled Custom Loss: {scaled_custom_loss.item()}, MSE Loss: {mse_loss.item()}")

            scaled_custom_loss.backward(retain_graph=True)


            #for name, param in self.model.named_parameters():
            #    if param.grad is not None:
            #        print(f"Gradient for {name}: {param.grad.norm()}")

            self.optimizer.step()
            loss_list.append(scaled_custom_loss.item())

            print(f"Epoch {epoch + 1}/{self.num_epochs}, Loss: {scaled_custom_loss.item()}")

In [26]:
class AngularLoss(nn.Module):
    def __init__(self):
        super(AngularLoss, self).__init__()

    def forward(self, predicted_angles, angles_tensor):
        predicted_angles_phi, predicted_angles_psi = predicted_angles[:, 0], predicted_angles[:, 1]
        angles_tensor_phi, angles_tensor_psi = angles_tensor[:, 0], angles_tensor[:, 1]

        predicted_angles_phi = (predicted_angles_phi + torch.pi) % (2 * torch.pi) - torch.pi
        angles_tensor_phi = (angles_tensor_phi + torch.pi) % (2 * torch.pi) - torch.pi
        predicted_angles_psi = (predicted_angles_psi + torch.pi) % (2 * torch.pi) - torch.pi
        angles_tensor_psi = (angles_tensor_psi + torch.pi) % (2 * torch.pi) - torch.pi

        difference_phi = torch.abs(predicted_angles_phi - angles_tensor_phi)
        loss_phi = torch.mean(torch.min(difference_phi, 2 * torch.pi - difference_phi))

        difference_psi = torch.abs(predicted_angles_psi - angles_tensor_psi)
        loss_psi = torch.mean(torch.min(difference_psi, 2 * torch.pi - difference_psi))

        loss = loss_phi+ loss_psi
        print(loss)
        return loss

class TransformerTrainer5:
    def __init__(self, model: nn.Module, criterion: nn.Module, num_epochs: int, sequence: torch.Tensor, angles: torch.Tensor):
        self.model = model
        self.criterion = criterion
        self.num_epochs = num_epochs
        self.sequence = sequence
        self.angles_tensor = angles
        self.optimizer = optim.AdamW(model.parameters(), lr=0.001)
        #self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.1)

    def train(self):
        loss_list = []
        for epoch in range(self.num_epochs):
            self.optimizer.zero_grad()
            predictions = self.model.forward(self.sequence)[:, :len(self.angles_tensor)]
            #loss = CustomLoss()
            #loss.forward(torch.tensor(predictions, requires_grad=True), self.angles_tensor)
            loss = self.criterion(predictions.squeeze(), self.angles_tensor)
            loss.backward(retain_graph=True)
            self.optimizer.step()
            #self.scheduler.step()
            loss_list.append(loss.item())
            #print(f"Epoch {epoch + 1}/{self.num_epochs}, Loss: {loss.item()}")
        return loss



In [27]:
bert_model = BertModel.from_pretrained("Rostlab/prot_bert")
output = bert_model(**encoded_input)
embedded_pb = output.last_hidden_state
N, D = embedded_pb.size()[1], embedded_pb.size()[2]
embeddings = embedded_pb
# Initialize and train transformer model
feed_forward_dim1 = 512
feed_forward_dim2 = 256
feed_forward_dim3 = 128
num_epochs = 20
model = TransformerModel(embed_dim=D, feed_forward_dim1=feed_forward_dim1, feed_forward_dim2= feed_forward_dim2)
#model.forward(embeddings)
criterion = MeanAngleErrorLoss()
num_epochs = 20
#criterion = nn.MSELoss()
#criterion = nn.L1Loss()

#criterion  = AngularLoss()
trainer = TransformerTrainer4(model, criterion, num_epochs, embedded_pb, angles)
trainer.train()
model

pytorch_model.bin:  12%|#2        | 210M/1.68G [00:00<?, ?B/s]

NameError: name 'encoded_input' is not defined